# **Memory & Chain**

## **1.환경준비**

### (1) 라이브러리 Import

In [1]:
import pandas as pd
import numpy as np
import os
import openai

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### (2) OpenAI API Key 확인
* 환경변수로 등록된 Key 확인하기

In [2]:
# 환경변수에서 키 불러오기
api_key = os.getenv('OPENAI_API_KEY')
print(api_key) 

sk-proj-7delVQofEYIm8iIiAvAdT3BlbkFJ7zw4gMd2DTBY5fq9XjT4


* 만약 환경변수 키 설정이 잘 안된다면 아래 코드셀의 주석을 해제하고, 자신의 api key를 입력하고 실행
    * 아래 코드는 키 지정을 **임시**로 수행함.
    * 파이썬 파일(.ipynb, .py)안에서 매번 수행해야 함.

In [62]:
# os.environ['OPENAI_API_KEY'] = '여러분의 OpenAI API키' 
# openai.api_key = os.getenv('OPENAI_API_KEY')

## **2.Memory**

### (1) 대화 기록 저장하기

In [3]:
# 메모리 선언하기(초기화)
memory = ConversationBufferMemory(return_messages=True) 

# 저장
memory.save_context({"input": "안녕하세요!"},
                    {"output": "안녕하세요! 어떻게 도와드릴까요?"})

memory.save_context({"input": "메일을 써야하는데 도와줘"},
                    {"output": "누구에게 보내는 어떤 메일인가요?"})

# 현재 담겨 있는 메모리 내용 전체 확인
memory.load_memory_variables({})

{'history': [HumanMessage(content='안녕하세요!'),
  AIMessage(content='안녕하세요! 어떻게 도와드릴까요?'),
  HumanMessage(content='메일을 써야하는데 도와줘'),
  AIMessage(content='누구에게 보내는 어떤 메일인가요?')]}

### (2) GPT와 대화 기록 저장하기

#### 1) 대화 준비

In [4]:
k = 3

# Chroma 데이터베이스 인스턴스
embeddings = OpenAIEmbeddings(model = "text-embedding-ada-002")
database = Chroma(persist_directory = "./data3", embedding_function = embeddings)

# retriever 선언
retriever = database.as_retriever(search_kwargs={"k": k})

# ChatOpenAI 선언
chat = ChatOpenAI(model="gpt-3.5-turbo")

# RetrievalQA 선언
qa = RetrievalQA.from_llm(llm=chat,  retriever=retriever,  return_source_documents=True)

#### 2) 대화 시도 및 기록

In [5]:
# 질문 답변1
query = "생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?"
result = qa(query)
result['result']

'생성형 AI를 도입할 때 예상되는 보안 위협은 다양합니다. 예를 들어, 생성형 AI가 악용되어 사적인 정보를 탈취하거나 위조된 콘텐츠를 생성하는 데 사용될 수 있습니다. 또한 생성형 AI가 허위 정보를 만들어내어 사람들을 혼란시키거나 속일 수도 있습니다. 또한 모델이 부적절하게 사용되어 편향된 결과를 도출하는 경우도 있을 수 있습니다. 이러한 위협들을 예방하기 위해 보안 및 규제 조치를 강화하는 것이 중요합니다.'

In [6]:
memory = ConversationBufferMemory(return_messages=True) 

memory.save_context({"input": query},
                    {"output": result['result']})

memory.load_memory_variables({})

{'history': [HumanMessage(content='생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?'),
  AIMessage(content='생성형 AI를 도입할 때 예상되는 보안 위협은 다양합니다. 예를 들어, 생성형 AI가 악용되어 사적인 정보를 탈취하거나 위조된 콘텐츠를 생성하는 데 사용될 수 있습니다. 또한 생성형 AI가 허위 정보를 만들어내어 사람들을 혼란시키거나 속일 수도 있습니다. 또한 모델이 부적절하게 사용되어 편향된 결과를 도출하는 경우도 있을 수 있습니다. 이러한 위협들을 예방하기 위해 보안 및 규제 조치를 강화하는 것이 중요합니다.')]}

#### 3) 이어지는 질문과 답변

In [7]:
# 질문 답변2
query = "훈련 데이터나 가중치를 오염시키는게 무슨 의미야?"
result = qa(query)
result['result']

memory.save_context({"input": query},
                    {"output": result['result']})

memory.load_memory_variables({})

{'history': [HumanMessage(content='생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?'),
  AIMessage(content='생성형 AI를 도입할 때 예상되는 보안 위협은 다양합니다. 예를 들어, 생성형 AI가 악용되어 사적인 정보를 탈취하거나 위조된 콘텐츠를 생성하는 데 사용될 수 있습니다. 또한 생성형 AI가 허위 정보를 만들어내어 사람들을 혼란시키거나 속일 수도 있습니다. 또한 모델이 부적절하게 사용되어 편향된 결과를 도출하는 경우도 있을 수 있습니다. 이러한 위협들을 예방하기 위해 보안 및 규제 조치를 강화하는 것이 중요합니다.'),
  HumanMessage(content='훈련 데이터나 가중치를 오염시키는게 무슨 의미야?'),
  AIMessage(content='훈련 데이터나 가중치를 오염시킨다는 것은 해당 데이터나 모델의 정보를 왜곡하거나 변형시키는 것을 의미합니다. 이는 모델의 학습이나 성능에 부정적인 영향을 미칠 수 있습니다.')]}

In [8]:
# 질문 답변3
query = "이를 방지하기 위해 어떻게 해야 해?"
result = qa(query)
result['result']

memory.save_context({"input": query},
                    {"output": result['result']})

memory.load_memory_variables({})

{'history': [HumanMessage(content='생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?'),
  AIMessage(content='생성형 AI를 도입할 때 예상되는 보안 위협은 다양합니다. 예를 들어, 생성형 AI가 악용되어 사적인 정보를 탈취하거나 위조된 콘텐츠를 생성하는 데 사용될 수 있습니다. 또한 생성형 AI가 허위 정보를 만들어내어 사람들을 혼란시키거나 속일 수도 있습니다. 또한 모델이 부적절하게 사용되어 편향된 결과를 도출하는 경우도 있을 수 있습니다. 이러한 위협들을 예방하기 위해 보안 및 규제 조치를 강화하는 것이 중요합니다.'),
  HumanMessage(content='훈련 데이터나 가중치를 오염시키는게 무슨 의미야?'),
  AIMessage(content='훈련 데이터나 가중치를 오염시킨다는 것은 해당 데이터나 모델의 정보를 왜곡하거나 변형시키는 것을 의미합니다. 이는 모델의 학습이나 성능에 부정적인 영향을 미칠 수 있습니다.'),
  HumanMessage(content='이를 방지하기 위해 어떻게 해야 해?'),
  AIMessage(content='어떤 것을 방지하려는지 구체적으로 알려주시면 도와드릴 수 있습니다.')]}

* 훈련 데이터 오염을 방지하기 위한 대책을 물었으나, 일반적인 보안 위협 방지 대책을 이야기 함.
* 맥락을 유지하기 위해서 메시지의 내용을 프롬프트에 포함시켜야 함.
    * 이를 손쉽게 엮어주는 방법 **Chain**

## **3.Chain**

* 절차 다시 정리
    * 질문을 받아
    * 유사도 높은 문서를 DB에서 검색(RAG)
    * 이전 대화 내용을 메모리에서 읽어오기
    * [질문 + 유사도높은 문서 + 이전 대화내용]으로 프롬프트 구성
    * GPT에 질문하고 답변 받기
* Chain
    * 이러한 절차를 코드로 하나하나 엮고, 프롬프트를 구성하는 코드는 상당히 복잡합니다.
    * 랭체인에서 제공하는 Chain 함수를 이용하면 쉽게 구현 가능!  

### (1) Chain 함수로 연결하기
* **ConversationalRetrievalChain**

In [9]:
embeddings = OpenAIEmbeddings(model = "text-embedding-ada-002")
database = Chroma(persist_directory = "./data3", embedding_function = embeddings)
chat = ChatOpenAI(model="gpt-3.5-turbo")

k=3
retriever = database.as_retriever(search_kwargs={"k": k})

# 대화 메모리 생성
memory = ConversationBufferMemory(memory_key="chat_history", input_key="question", output_key="answer", 
                                  return_messages=True)

# ConversationalRetrievalQA 체인 생성
qa = ConversationalRetrievalChain.from_llm(llm=chat, retriever=retriever, memory=memory, 
                                           return_source_documents=True,  output_key="answer")

### (2) 사용하기

#### 1) 첫번째 질문

In [10]:
# 첫번째 질문
query1 = "생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?"
result = qa(query1)
result['answer']

'생성형 AI를 도입할 때 예상되는 보안 위협에는 다음과 같은 것들이 있을 수 있습니다: \n1. 악의적인 사용: 생성형 AI가 악의적인 목적으로 사용될 수 있으며, 예를 들어 가짜 뉴스나 위조된 정보 생성에 활용될 수 있습니다. \n2. 개인정보 노출: 생성형 AI가 개인정보를 수집하거나 처리하는 과정에서 보안 위험이 발생할 수 있습니다. \n3. 모델 오용: 생성형 AI 모델이 오용되어 부적절한 결과를 생성하거나 악용될 수 있습니다. \n4. 데이터 유출: 생성형 AI를 학습시키는 데이터가 유출되면 보안 위험이 발생할 수 있습니다. \n5. 공격 및 변조: 생성형 AI 시스템 자체가 해킹이나 악의적인 조작을 당할 수 있습니다.\n\n더 자세한 정보를 얻기 위해서는 보안 전문가나 관련 전문가들의 조언을 구하는 것이 좋습니다.'

In [11]:
# 메모리 확인
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?'),
  AIMessage(content='생성형 AI를 도입할 때 예상되는 보안 위협에는 다음과 같은 것들이 있을 수 있습니다: \n1. 악의적인 사용: 생성형 AI가 악의적인 목적으로 사용될 수 있으며, 예를 들어 가짜 뉴스나 위조된 정보 생성에 활용될 수 있습니다. \n2. 개인정보 노출: 생성형 AI가 개인정보를 수집하거나 처리하는 과정에서 보안 위험이 발생할 수 있습니다. \n3. 모델 오용: 생성형 AI 모델이 오용되어 부적절한 결과를 생성하거나 악용될 수 있습니다. \n4. 데이터 유출: 생성형 AI를 학습시키는 데이터가 유출되면 보안 위험이 발생할 수 있습니다. \n5. 공격 및 변조: 생성형 AI 시스템 자체가 해킹이나 악의적인 조작을 당할 수 있습니다.\n\n더 자세한 정보를 얻기 위해서는 보안 전문가나 관련 전문가들의 조언을 구하는 것이 좋습니다.')]}

#### 2) 두번째 질문

In [12]:
# 두번째 질문
query2 = "모델을 재학습시면 어떤 문제가 발생되는거야?"
result = qa(query2)
result['answer']

'모델을 재학습시킬 때 보안 문제가 발생할 수 있습니다. 예를 들어, 모델을 재학습시키는 동안 데이터 유출이 발생할 수 있고, 이를 악용하여 모델의 성능을 악용하는 공격이 가능합니다. 또한 모델을 업데이트하는 과정에서 보안 취약점이 발생할 수도 있습니다. 보안을 고려하여 모델 재학습을 진행해야 합니다.'

In [13]:
# 메모리 확인
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?'),
  AIMessage(content='생성형 AI를 도입할 때 예상되는 보안 위협에는 다음과 같은 것들이 있을 수 있습니다: \n1. 악의적인 사용: 생성형 AI가 악의적인 목적으로 사용될 수 있으며, 예를 들어 가짜 뉴스나 위조된 정보 생성에 활용될 수 있습니다. \n2. 개인정보 노출: 생성형 AI가 개인정보를 수집하거나 처리하는 과정에서 보안 위험이 발생할 수 있습니다. \n3. 모델 오용: 생성형 AI 모델이 오용되어 부적절한 결과를 생성하거나 악용될 수 있습니다. \n4. 데이터 유출: 생성형 AI를 학습시키는 데이터가 유출되면 보안 위험이 발생할 수 있습니다. \n5. 공격 및 변조: 생성형 AI 시스템 자체가 해킹이나 악의적인 조작을 당할 수 있습니다.\n\n더 자세한 정보를 얻기 위해서는 보안 전문가나 관련 전문가들의 조언을 구하는 것이 좋습니다.'),
  HumanMessage(content='모델을 재학습시면 어떤 문제가 발생되는거야?'),
  AIMessage(content='모델을 재학습시킬 때 보안 문제가 발생할 수 있습니다. 예를 들어, 모델을 재학습시키는 동안 데이터 유출이 발생할 수 있고, 이를 악용하여 모델의 성능을 악용하는 공격이 가능합니다. 또한 모델을 업데이트하는 과정에서 보안 취약점이 발생할 수도 있습니다. 보안을 고려하여 모델 재학습을 진행해야 합니다.')]}

### (3) 반복문 안에서 질문답변 이어가기

In [14]:
while True:
    query = input('질문 > ')
    query = query.strip()
    print(f'질문 : {query}')
    print('-' * 20)
    if len(query) == 0:
        break
    result = qa({"question": query})
    print(f'답변 : {result["answer"]}')
    print('=' * 50)

질문 : 안녕
--------------------
답변 : 안녕하세요. 아직 무엇을 해드릴 수 있을까요?
질문 : 답변해줘
--------------------
답변 : 네, 질문이 무엇인가요?
질문 : ai 문제가 있을까
--------------------
답변 : 네, AI 시스템에는 여러 가지 문제가 발생할 수 있습니다. 예를 들어, 데이터의 품질이 낮거나 편향된 경우 AI 모델의 성능이 저하될 수 있고, 모델이 잘못된 결정을 내리거나 예측을 하기도 합니다. 또한, AI 모델이 사용자의 개인 정보를 부적절하게 다루거나 윤리적인 문제를 야기할 수도 있습니다. 이러한 문제들을 해결하기 위해서는 적절한 모델 개발 및 윤리적인 사용이 필요합니다.
질문 : 잘못된 예측의 예를 알려줘
--------------------
답변 : 죄송합니다. 제가 알고 있는 정보로는 구체적인 예를 드릴 수 없습니다.
질문 : 
--------------------
